In [1]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data1 = pd.read_excel('治疗方案.xlsx') # 治疗方案
data2 = pd.read_excel('含分组信息数据_血肿.xlsx') # 血肿数据

In [3]:
data3 = pd.merge(data1, data2, how='outer', on='ID')
data3 = data3.drop(range(101, 131))

In [4]:
data3.columns

Index(['ID', '脑室引流_x', '止血治疗_x', '降颅压治疗_x', '降压治疗_x', '镇静、镇痛治疗_x', '止吐护胃_x',
       '营养神经_x', '脑室引流_y', '止血治疗_y', '降颅压治疗_y', '降压治疗_y', '镇静、镇痛治疗_y',
       '止吐护胃_y', '营养神经_y', '入院首次影像检查流水号', 'HM_volume', '随访1流水号', 'HM_volume.1',
       '随访2流水号', 'HM_volume.2', '随访3流水号', 'HM_volume.3', '随访4流水号',
       'HM_volume.4', '随访5流水号', 'HM_volume.5', '随访6流水号', 'HM_volume.6',
       '随访7流水号', 'HM_volume.7', '随访8流水号', 'HM_volume.8', '分组信息'],
      dtype='object')

In [5]:
import json
with open('best_labels_130.json','r') as f:
    groups_label=json.load(f)
print(len(groups_label['best_labels_130']))

130


In [6]:
dataT=data3
dataT['分组信息']=groups_label['best_labels_130']
dataT.to_excel('含分组信息数据.xlsx',index=False)

In [7]:
# 创建一个新列"差"，初始化为0
dataT['差'] = 0
U=globals()
res={}
for i in list(set(dataT['分组信息'])):
    U[f'data_g{i}']=dataT[dataT['分组信息']==i]

# 遍历数据集的每一行
    for index, row in U[f'data_g{i}'].iterrows():
        row_sum = 0  # 用于累积每行的差值

        prev_value = None  # 用于存储前一个以"ED"开头的列的值

        # 遍历以"ED"开头的列
        for col in row.index:
            if col.startswith('ED'):
                value = row[col]
                if not pd.isna(value):  # 确保元素不是NaN
                    if prev_value is not None:
                        row_sum += (value - prev_value)  # 计算差值并累积
                    prev_value = value

        # 根据row_sum的值设置"差"列的值
        if row_sum > 0:
            U[f'data_g{i}'].at[index, '差'] = 1
        elif row_sum < 0:
            U[f'data_g{i}'].at[index, '差'] = -1
        res[f'组{i}']=float(U[f'data_g{i}']['差'].sum())

res_res = {}
for i in res:
    if res[i] < 0:
        res_res[i]=-1
    else:
        res_res[i]=1
# with open('分组趋势.json', 'w') as f:
#     json.dump(res_res,f)

In [8]:
# with open('分组趋势.json', 'r') as f:
#     print(json.load(f))

In [9]:
res_res

{'组0': 1, '组1': 1, '组2': 1, '组3': 1, '组4': 1}

In [10]:
#################### 合并最终表

In [11]:
data4 = pd.read_excel("治疗方案.xlsx")
data4 = data4.drop(range(101, 131))
mapping_dict = {}
p = 1
for col in data4.columns:
    # 如果列名包含[符号，则进行转换
    if '[' in col and ']' in col:
        mapping_dict[f'治疗组合{p}'] = col  # 添加映射关系到字典
        # 更新DataFrame的列名
        data4.rename(columns={col: f'治疗组合{p}'}, inplace=True)
        p += 1
with open('mapping.json', 'w') as json_file:
    json.dump(mapping_dict, json_file, ensure_ascii=False)
# data4.columns
data4.head()

,ID,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']"
0,sub001,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
1,sub002,1,1,1,1,1,0,1,0,1,...,1,1,0,0,0,0,0,0,0,0
2,sub003,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
3,sub004,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,sub005,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
target = []
for count, key in enumerate(groups_label['best_labels_130']):
    value = res_res['组'+str(key)]
    target.append(value)

In [13]:
data4['组趋势'] = 0
data4['组趋势'] = np.array(target)
data4 = data4.drop(['ID'], axis=1)
data4.head()

,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']","['止血治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']",组趋势
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [14]:
import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data4.drop('组趋势', axis=1)
Y = data4['组趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗血肿_regression_summary_组合_组趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                    组趋势   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -4.160
Date:                Sun, 24 Sep 2023   Prob (F-statistic):               1.00
Time:                        16:19:56   Log-Likelihood:                 3979.9
No. Observations:                 130   AIC:                            -7908.
Df Residuals:                     104   BIC:                            -7833.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

In [15]:
# 做单一治疗方法的
data5 = pd.concat([data1, data4[['组趋势']]], axis=1).drop(['ID'], axis=1)[:100]
data5

,脑室引流,止血治疗,降颅压治疗,降压治疗,镇静、镇痛治疗,止吐护胃,营养神经,组趋势
0,0,1,1,1,1,1,1,1.0
1,0,1,1,1,0,1,1,1.0
2,0,1,1,1,1,1,1,1.0
3,0,1,1,1,0,0,0,1.0
4,0,1,1,0,0,1,1,1.0
...,...,...,...,...,...,...,...,...
95,0,0,0,1,1,1,1,1.0
96,0,1,1,1,1,1,1,1.0
97,0,1,1,1,1,1,1,1.0
98,1,1,1,1,1,1,1,1.0


In [16]:
import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('组趋势', axis=1)
Y = data5['组趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗血肿_regression_summary_单一_组趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                    组趋势   R-squared:                        -inf
Model:                            OLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -13.14
Date:                Sun, 24 Sep 2023   Prob (F-statistic):               1.00
Time:                        16:19:56   Log-Likelihood:                 3314.8
No. Observations:                 100   AIC:                            -6614.
Df Residuals:                      92   BIC:                            -6593.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0000   7.78e-16   1.29e+15      0.0

In [17]:
singel_target = []
for index, row in pd.read_excel('含分组信息数据.xlsx').iterrows():
    t = 0
    row_sum = 0  # 用于累积每行的差值
    prev_value = None  # 用于存储前一个以"ED"开头的列的值
    # 遍历以"ED"开头的列
    for col in row.index:
        if col.startswith('ED'):
            value = row[col]
            if not pd.isna(value):  # 确保元素不是NaN
                if prev_value is not None:
                    row_sum += (value - prev_value)  # 计算差值并累积
                prev_value = value
    if row_sum > 0:
        t = 1
    elif row_sum < 0:
        t = -1
    singel_target.append(t)

In [18]:
# 做单一趋势的
data5 = pd.concat([data1, pd.DataFrame(np.array(singel_target), columns = ['个人趋势'])], axis=1).drop(['ID'], axis=1)[:100]
data5

,脑室引流,止血治疗,降颅压治疗,降压治疗,镇静、镇痛治疗,止吐护胃,营养神经,个人趋势
0,0,1,1,1,1,1,1,0.0
1,0,1,1,1,0,1,1,0.0
2,0,1,1,1,1,1,1,0.0
3,0,1,1,1,0,0,0,0.0
4,0,1,1,0,0,1,1,0.0
...,...,...,...,...,...,...,...,...
95,0,0,0,1,1,1,1,0.0
96,0,1,1,1,1,1,1,0.0
97,0,1,1,1,1,1,1,0.0
98,1,1,1,1,1,1,1,0.0


In [19]:
# 做单独治疗无组合+个人趋势的

import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('个人趋势', axis=1)
Y = data5['个人趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗血肿_regression_summary_单一_个人趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                   个人趋势   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 24 Sep 2023   Prob (F-statistic):                nan
Time:                        16:19:56   Log-Likelihood:                    inf
No. Observations:                 100   AIC:                              -inf
Df Residuals:                      92   BIC:                              -inf
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        n

In [20]:
data5 = pd.concat([data4.drop(['组趋势'],axis=1),pd.DataFrame(np.array(singel_target), columns = ['个人趋势'])], axis = 1)[:100]
data5

,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']","['止血治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']",个人趋势
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
97,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
98,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [21]:
# 做组合+个人趋势的

import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('个人趋势', axis=1)
Y = data5['个人趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗血肿_regression_summary_组合_个人趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                   个人趋势   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 24 Sep 2023   Prob (F-statistic):                nan
Time:                        16:19:57   Log-Likelihood:                    inf
No. Observations:                 100   AIC:                              -inf
Df Residuals:                      82   BIC:                              -inf
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------